In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp
import sklearn.model_selection 
import pandas as pd


# stochastic gradient descent (SGD)
using batches to fasten gradient computation (approximation)

SGD has to be implemented with static stepsize

In [28]:
def sigmoid(x): #giusto perforza
    return 1/(1-np.exp(-x))

def f(w, x): #giusto, ma bigìsogna capire chi fa xhat
    #xhat=np.concatenate((x, np.ones((1, x.shape[1]))), axis=0)
    return sigmoid(x.T@w)

def MSE(y, y1): #giusto, bisogna passare f come y e label(Y) come y1
    return (np.linalg.norm(y-y1)**2)/y.size[0]

def l(w,D): # giusto
    X,Y=D
    return (MSE(f(w,X), Y))

#gradient of l function grad_l
def grad_l(w, D):
    #D is the current batch: a tuple (Mx, My)
    #Mx is d x M
    #My is M
    Mx, My=D
    d, N=Mx.shape
    return (sigmoid(Mx.T@w).T@(1-sigmoid(Mx.T@w))*Mx@(f(w,Mx)-My)).T

#gradient of SGD 
def grad(w, D):
    #D is the current batch: a tuple (Mx, My)
    #gradient of SGD (sommatoria dei gradienti degli indici del batch [subset M])
    #this is an approximation of the gradiant, it isn't for sure the direction of steepest descent
    Mx, My=D
    d, N=Mx.shape
    #sum of the gradients (grad_l) of the indexes of the batch
    return np.sum(grad_l(w, D), axis=1)

    


def SGD(l, grad_l, w0, D, batch_size, n_epochs): #D is a tuple (x,y)
    #initialization
    a=1 #fixed
    # D =(X,Y) where X is d x N 
    # Y is N
    X, Y= D
    d,N=X.shape
    n_batch_per_epoch=int(N/batch_size)
    
    #definitions
    f_val=np.zeros((n_epochs, N))
    x_val=np.zeros((n_epochs, N))
    grads=np.zeros((n_epochs, N))
    err=np.zeros((n_epochs, N))
    w=w0
    i=0
    batchInd=np.zeros(batch_size)
    
    #loop
    conditions=True
    X_backup=X
    Y_backup=Y
    shuffledInd=np.arange(N)
    print(shuffledInd.shape)

    for epoch in range(n_epochs):
        #NOTA: YOU HAVE TO SHUFFLE AGAIN AFTER EACH EPOCH!!!!
        print(f"epoch {epoch}/{n_epochs}")
        for k in range (n_batch_per_epoch):
            print(f"batch {k}/{n_batch_per_epoch}")
            #sample M from D
            batchInd=shuffledInd[batch_size*k : batch_size*(k+1)]
            Mx=X[:,batchInd] #batch di X
            My=Y[batchInd]#batch di Y
            M=(Mx, My)
            #remove Mx and My from X and Y
            X=np.delete(X, batchInd, axis=1)
            Y=np.delete(Y, batchInd, axis=0)
            #update w
            w=w0-a*grad(w,M)
            #assignments
            f_val[epoch,i]=l(w, M)
            x_val[epoch, i]=w
            grads[epoch, i]=grad(w,M)
            err[epoch, i]=MSE(f(w, Mx), My)
            #restart 
            w0=w
            i+=1
        #reload X and Y and restart
        X=X_backup
        Y=Y_backup
        i=0
        shuffledInd=np.random.shuffle(np.arange(N)) #shuffle indexes

         
    return f_val, x_val, grads, err
 #REMEMBER: w0 in SGD should be chosen randomly (sample from Gaussian), and not fixed
 #(upgrade): modify the code above to return f_val (over M)
 #f_val[k]=l(w_k, M)




In [29]:
Data=pd.read_csv("../2/data.csv")
#print(D.head())
Data=np.array(Data)
X=Data[:,1:]
Y=Data[:,0]
#isolate digits 3 and 1
i1= (Y==1)
i3= (Y==3)
X=X[i1+i3,:]
Y=Y[i1+i3]
N,d=X.shape
Y=Y.reshape((N, 1))
N,d=X.shape
X_train, X_test, y_train, y_test= sklearn.model_selection.train_test_split(X, Y, test_size=0.33, random_state=1)
X=X.T
X_train=X_train.T
X_test=X_test.T
print(f"shape of X: {X.shape}")
#call SGD
#initialize w0 with normal distribution
w0=np.random.normal(0,1,(d,1))
D=(X_train, y_train)
f_val, x_val, grads, err=SGD(l, grad_l, w0, D, batch_size=100, n_epochs=10)
 

shape of X: (784, 9035)
(6053,)
epoch 0/10
batch 0/60


C:\Users\gnele\AppData\Local\Temp\ipykernel_10336\2469562586.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1-np.exp(-x))


TypeError: 'int' object is not subscriptable

In [27]:
M=(X_test, y_test)
test=(sigmoid(X_test.T@w0).T@(1-sigmoid(X_test.T@w0))*X_test@(f(w0,X_test)-y_test)).T
print(test.shape)
#print(grad_l(w0, M))

(1, 784)


C:\Users\gnele\AppData\Local\Temp\ipykernel_10336\1521652765.py:2: RuntimeWarning: overflow encountered in exp
  return 1/(1-np.exp(-x))
